# 5×5 Tic Tac Toe (GUI App) + Dynamic Obstacles + AI (Minimax + Alpha-Beta)

✅ Running this notebook will open a **separate Tkinter window** (app-like game board).

Features:
- 5×5 board
- Win = 4-in-a-row (K=4)
- Start with 2 random obstacles (#)
- Dynamic obstacles: every 4 moves (late game: every 6 moves)
- Fairness:
  - no obstacle adjacent to the last move (8-direction)
  - obstacle won’t block any immediate winning move
- AI: Minimax + Alpha-Beta + depth limit + heuristic (with light sampling on obstacle turns)

> If you're on Windows/macOS/Linux locally, Tkinter should open a new window.
> If you're on Google Colab / web-only notebooks, Tkinter windows usually won't open.


In [1]:
# Configuration (you can tweak these)
AI_DEPTH = 3           # 3 = smooth, 4 = stronger but slower
K_TO_WIN = 4           # 4-in-a-row
START_OBSTACLES = 2    # initial obstacles
SAMPLE_OBSTACLES = 6   # obstacle randomness sampling inside minimax (bigger = smarter but slower)


In [2]:

import random
import math
from copy import deepcopy

SIZE = 5
EMPTY = " "
OBSTACLE = "#"

def create_board():
    return [[EMPTY for _ in range(SIZE)] for _ in range(SIZE)]

def get_empty_cells(board):
    return [(r, c) for r in range(SIZE) for c in range(SIZE) if board[r][c] == EMPTY]

def is_draw(board):
    return len(get_empty_cells(board)) == 0

def get_adjacent_cells(r, c):
    adj = set()
    for dr in (-1, 0, 1):
        for dc in (-1, 0, 1):
            if dr == 0 and dc == 0:
                continue
            rr, cc = r + dr, c + dc
            if 0 <= rr < SIZE and 0 <= cc < SIZE:
                adj.add((rr, cc))
    return adj

def check_win(board, player, K=4):
    dirs = [(0,1), (1,0), (1,1), (1,-1)]
    for r in range(SIZE):
        for c in range(SIZE):
            if board[r][c] != player:
                continue
            for dr, dc in dirs:
                cnt = 0
                rr, cc = r, c
                while 0 <= rr < SIZE and 0 <= cc < SIZE and board[rr][cc] == player:
                    cnt += 1
                    if cnt >= K:
                        return True
                    rr += dr
                    cc += dc
    return False

def winning_moves(board, player, K=4):
    wins = set()
    for r, c in get_empty_cells(board):
        board[r][c] = player
        if check_win(board, player, K=K):
            wins.add((r, c))
        board[r][c] = EMPTY
    return wins

def obstacle_frequency(board):
    empty = len(get_empty_cells(board))
    return 6 if empty <= 8 else 4

def add_obstacle_fair(board, last_move=None, K=4, players=("X", "O")):
    empties = get_empty_cells(board)
    if not empties:
        return False

    win_block = set()
    for p in players:
        win_block |= winning_moves(board, p, K=K)

    adj_forbidden = set()
    if last_move is not None:
        adj_forbidden = get_adjacent_cells(*last_move)

    # Step A: avoid adjacency + avoid win-block
    cand = [cell for cell in empties if cell not in adj_forbidden and cell not in win_block]
    if cand:
        r, c = random.choice(cand)
        board[r][c] = OBSTACLE
        return (r, c)

    # Step B: allow adjacency, still avoid win-block
    cand = [cell for cell in empties if cell not in win_block]
    if cand:
        r, c = random.choice(cand)
        board[r][c] = OBSTACLE
        return (r, c)

    # Step C: avoid adjacency, allow win-block
    cand = [cell for cell in empties if cell not in adj_forbidden]
    if cand:
        r, c = random.choice(cand)
        board[r][c] = OBSTACLE
        return (r, c)

    # Step D: fallback any empty cell
    r, c = random.choice(empties)
    board[r][c] = OBSTACLE
    return (r, c)

def init_obstacles(board, n=2, K=4):
    for _ in range(n):
        add_obstacle_fair(board, last_move=None, K=K)

# ---------------------------
# Heuristic + Minimax AB
# ---------------------------
def evaluate_window(window, ai, human):
    if OBSTACLE in window:
        return 0

    ai_count = window.count(ai)
    hu_count = window.count(human)
    empty_count = window.count(EMPTY)

    if ai_count > 0 and hu_count > 0:
        return 0

    if ai_count == 4:
        return 100000
    if hu_count == 4:
        return -100000

    score = 0
    if ai_count == 3 and empty_count == 1:
        score += 200
    elif ai_count == 2 and empty_count == 2:
        score += 30
    elif ai_count == 1 and empty_count == 3:
        score += 5

    if hu_count == 3 and empty_count == 1:
        score -= 220
    elif hu_count == 2 and empty_count == 2:
        score -= 35
    elif hu_count == 1 and empty_count == 3:
        score -= 6

    return score

def heuristic(board, ai="O", human="X", K=4):
    score = 0

    # rows
    for r in range(SIZE):
        for c in range(SIZE - K + 1):
            window = [board[r][c+i] for i in range(K)]
            score += evaluate_window(window, ai, human)

    # cols
    for c in range(SIZE):
        for r in range(SIZE - K + 1):
            window = [board[r+i][c] for i in range(K)]
            score += evaluate_window(window, ai, human)

    # diag down-right
    for r in range(SIZE - K + 1):
        for c in range(SIZE - K + 1):
            window = [board[r+i][c+i] for i in range(K)]
            score += evaluate_window(window, ai, human)

    # diag down-left
    for r in range(SIZE - K + 1):
        for c in range(K - 1, SIZE):
            window = [board[r+i][c-i] for i in range(K)]
            score += evaluate_window(window, ai, human)

    return score

def immediate_tactics_order(board, ai, human, K=4):
    empties = get_empty_cells(board)
    ai_wins = winning_moves(board, ai, K=K)
    hu_wins = winning_moves(board, human, K=K)

    ordered = []
    for m in empties:
        if m in ai_wins:
            ordered.append(m)
    for m in empties:
        if m in hu_wins and m not in ordered:
            ordered.append(m)
    for m in empties:
        if m not in ordered:
            ordered.append(m)
    return ordered

def minimax_ab(board, depth, alpha, beta, maximizing, ai, human, K, total_moves, last_move, sample_obstacles=6):
    if check_win(board, ai, K=K):
        return 10**9
    if check_win(board, human, K=K):
        return -10**9
    if depth == 0 or is_draw(board):
        return heuristic(board, ai=ai, human=human, K=K)

    moves = immediate_tactics_order(board, ai, human, K=K)

    if maximizing:
        best = -math.inf
        for r, c in moves:
            if board[r][c] != EMPTY:
                continue
            b2 = deepcopy(board)
            b2[r][c] = ai
            new_total = total_moves + 1
            new_last = (r, c)

            freq = obstacle_frequency(b2)
            if new_total % freq == 0:
                vals = []
                for _ in range(sample_obstacles):
                    b3 = deepcopy(b2)
                    add_obstacle_fair(b3, last_move=new_last, K=K, players=(human, ai))
                    vals.append(minimax_ab(b3, depth-1, alpha, beta, False, ai, human, K, new_total, new_last, sample_obstacles))
                val = sum(vals) / len(vals)
            else:
                val = minimax_ab(b2, depth-1, alpha, beta, False, ai, human, K, new_total, new_last, sample_obstacles)

            best = max(best, val)
            alpha = max(alpha, best)
            if beta <= alpha:
                break
        return best
    else:
        best = math.inf
        for r, c in moves:
            if board[r][c] != EMPTY:
                continue
            b2 = deepcopy(board)
            b2[r][c] = human
            new_total = total_moves + 1
            new_last = (r, c)

            freq = obstacle_frequency(b2)
            if new_total % freq == 0:
                vals = []
                for _ in range(sample_obstacles):
                    b3 = deepcopy(b2)
                    add_obstacle_fair(b3, last_move=new_last, K=K, players=(human, ai))
                    vals.append(minimax_ab(b3, depth-1, alpha, beta, True, ai, human, K, new_total, new_last, sample_obstacles))
                val = sum(vals) / len(vals)
            else:
                val = minimax_ab(b2, depth-1, alpha, beta, True, ai, human, K, new_total, new_last, sample_obstacles)

            best = min(best, val)
            beta = min(beta, best)
            if beta <= alpha:
                break
        return best

def choose_ai_move(board, ai="O", human="X", depth=3, K=4, total_moves=0, last_move=None, sample_obstacles=6):
    best_val = -math.inf
    best_move = None

    for r, c in immediate_tactics_order(board, ai, human, K=K):
        if board[r][c] != EMPTY:
            continue
        b2 = deepcopy(board)
        b2[r][c] = ai
        val = minimax_ab(
            b2, depth-1, -math.inf, math.inf,
            False, ai, human, K,
            total_moves + 1, (r, c),
            sample_obstacles=sample_obstacles
        )
        if val > best_val:
            best_val = val
            best_move = (r, c)

    return best_move


In [3]:

import tkinter as tk
from tkinter import ttk

# =========================
# Colorful, optimized Canvas GUI
# =========================

PALETTE = {
    # Window / panels
    "bg": "#0B1020",
    "panel": "#111A33",
    "panel2": "#0F1730",
    "text": "#E6EAF2",
    "muted": "#A6B0C3",
    "border": "#2B3A74",
    "accent": "#00D1B2",

    # Board cells
    "cell_empty": "#17224A",
    "cell_hover": "#1D2B5C",
    "cell_x": "#2A1630",
    "cell_o": "#10254D",
    "cell_obs": "#2B240E",

    # Pieces
    "x": "#FF5C8A",
    "o": "#4D96FF",
    "obs": "#FFD93D",
    "win": "#00D1B2",
}


def find_winning_line(board, player, K=4):
    """Return list of (r,c) cells forming a winning K-in-a-row line, else None."""
    dirs = [(0, 1), (1, 0), (1, 1), (1, -1)]  # right, down, diag, anti-diag
    for r in range(SIZE):
        for c in range(SIZE):
            if board[r][c] != player:
                continue
            for dr, dc in dirs:
                path = [(r, c)]
                rr, cc = r + dr, c + dc
                while 0 <= rr < SIZE and 0 <= cc < SIZE and board[rr][cc] == player:
                    path.append((rr, cc))
                    if len(path) >= K:
                        return path[:K]
                    rr += dr
                    cc += dc
    return None


class TicTacToeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("5x5 Tic Tac Toe — Dynamic Obstacles + AI")
        self.root.configure(bg=PALETTE["bg"])
        self.root.resizable(False, False)

        # ttk theme
        style = ttk.Style()
        try:
            style.theme_use("clam")
        except Exception:
            pass

        style.configure("App.TFrame", background=PALETTE["bg"])
        style.configure("Card.TFrame", background=PALETTE["panel"], relief="flat")
        style.configure("Title.TLabel", background=PALETTE["bg"], foreground=PALETTE["text"], font=("Segoe UI", 18, "bold"))
        style.configure("Sub.TLabel", background=PALETTE["bg"], foreground=PALETTE["muted"], font=("Segoe UI", 10))
        style.configure("CardTitle.TLabel", background=PALETTE["panel"], foreground=PALETTE["text"], font=("Segoe UI", 12, "bold"))
        style.configure("CardText.TLabel", background=PALETTE["panel"], foreground=PALETTE["muted"], font=("Segoe UI", 10))
        style.configure("Status.TLabel", background=PALETTE["bg"], foreground=PALETTE["text"], font=("Segoe UI", 11))
        style.configure("App.TRadiobutton", background=PALETTE["panel"], foreground=PALETTE["text"], font=("Segoe UI", 10))

        # Header
        header = ttk.Frame(root, padding=(16, 14), style="App.TFrame")
        header.grid(row=0, column=0, columnspan=2, sticky="ew")
        ttk.Label(header, text="5×5 Tic Tac Toe", style="Title.TLabel").grid(row=0, column=0, sticky="w")
        ttk.Label(header, text="4-in-a-row wins • obstacles appear over time • AI uses minimax", style="Sub.TLabel").grid(row=1, column=0, sticky="w", pady=(2, 0))
        header.columnconfigure(0, weight=1)

        # Main layout
        main = ttk.Frame(root, padding=(16, 0, 16, 16), style="App.TFrame")
        main.grid(row=1, column=0, columnspan=2, sticky="nsew")

        # Board sizing
        self.cell_px = 80
        self.gap = 8
        self.step = self.cell_px + self.gap
        self.board_px = SIZE * self.cell_px + (SIZE - 1) * self.gap

        # Board card
        board_card = tk.Frame(main, bg=PALETTE["panel"], highlightthickness=2, highlightbackground=PALETTE["border"])
        board_card.grid(row=0, column=0, padx=(0, 14), pady=(14, 0))

        board_top = tk.Frame(board_card, bg=PALETTE["panel"], padx=12, pady=10)
        board_top.pack(fill="x")
        self.turn_dot = tk.Canvas(board_top, width=12, height=12, bg=PALETTE["panel"], highlightthickness=0)
        self.turn_dot.pack(side="left", padx=(0, 8))
        self.turn_dot_id = self.turn_dot.create_oval(2, 2, 10, 10, fill=PALETTE["x"], outline="")
        self.turn_text = tk.StringVar(value="Player X")
        tk.Label(board_top, textvariable=self.turn_text, bg=PALETTE["panel"], fg=PALETTE["text"], font=("Segoe UI", 12, "bold")).pack(side="left")

        self.canvas = tk.Canvas(
            board_card,
            width=self.board_px,
            height=self.board_px,
            bg=PALETTE["panel2"],
            highlightthickness=0,
        )
        self.canvas.pack(padx=12, pady=(0, 12))

        # Sidebar card
        side_card = tk.Frame(main, bg=PALETTE["panel"], highlightthickness=2, highlightbackground=PALETTE["border"], padx=12, pady=12)
        side_card.grid(row=0, column=1, sticky="n", pady=(14, 0))

        tk.Label(side_card, text="Settings", bg=PALETTE["panel"], fg=PALETTE["text"], font=("Segoe UI", 12, "bold")).grid(row=0, column=0, sticky="w")

        self.mode_var = tk.StringVar(value="AI")
        mode_box = tk.Frame(side_card, bg=PALETTE["panel"], pady=6)
        mode_box.grid(row=1, column=0, sticky="ew")
        ttk.Radiobutton(mode_box, text="Human vs AI", variable=self.mode_var, value="AI", style="App.TRadiobutton", command=self._sync_controls).grid(row=0, column=0, sticky="w")
        ttk.Radiobutton(mode_box, text="Human vs Human", variable=self.mode_var, value="HUMAN", style="App.TRadiobutton", command=self._sync_controls).grid(row=1, column=0, sticky="w")

        # Difficulty
        self.ai_depth_var = tk.IntVar(value=AI_DEPTH)
        tk.Label(side_card, text="AI strength (depth)", bg=PALETTE["panel"], fg=PALETTE["muted"], font=("Segoe UI", 10)).grid(row=2, column=0, sticky="w", pady=(6, 0))
        self.depth_scale = tk.Scale(
            side_card,
            from_=1,
            to=5,
            orient="horizontal",
            variable=self.ai_depth_var,
            bg=PALETTE["panel"],
            fg=PALETTE["text"],
            highlightthickness=0,
            troughcolor=PALETTE["panel2"],
            activebackground=PALETTE["accent"],
            length=220,
        )
        self.depth_scale.grid(row=3, column=0, sticky="ew")

        # Start obstacles
        self.start_obs_var = tk.IntVar(value=START_OBSTACLES)
        tk.Label(side_card, text="Starting obstacles", bg=PALETTE["panel"], fg=PALETTE["muted"], font=("Segoe UI", 10)).grid(row=4, column=0, sticky="w", pady=(8, 0))
        self.obs_scale = tk.Scale(
            side_card,
            from_=0,
            to=6,
            orient="horizontal",
            variable=self.start_obs_var,
            bg=PALETTE["panel"],
            fg=PALETTE["text"],
            highlightthickness=0,
            troughcolor=PALETTE["panel2"],
            activebackground=PALETTE["accent"],
            length=220,
        )
        self.obs_scale.grid(row=5, column=0, sticky="ew")

        # Buttons
        btn_row = tk.Frame(side_card, bg=PALETTE["panel"], pady=10)
        btn_row.grid(row=6, column=0, sticky="ew")
        self.new_btn = tk.Button(
            btn_row,
            text="New Game",
            command=self.reset_game,
            bg=PALETTE["accent"],
            fg=PALETTE["bg"],
            activebackground=PALETTE["accent"],
            activeforeground=PALETTE["bg"],
            relief="flat",
            bd=0,
            font=("Segoe UI", 10, "bold"),
            padx=14,
            pady=8,
        )
        self.new_btn.pack(side="left", padx=(0, 8))

        self.reset_score_btn = tk.Button(
            btn_row,
            text="Reset Scores",
            command=self.reset_scores,
            bg=PALETTE["panel2"],
            fg=PALETTE["text"],
            activebackground=PALETTE["panel2"],
            activeforeground=PALETTE["text"],
            relief="flat",
            bd=0,
            font=("Segoe UI", 10, "bold"),
            padx=14,
            pady=8,
        )
        self.reset_score_btn.pack(side="left")

        # Legend
        legend = tk.Frame(side_card, bg=PALETTE["panel"], pady=8)
        legend.grid(row=7, column=0, sticky="ew")
        tk.Label(legend, text="Legend", bg=PALETTE["panel"], fg=PALETTE["text"], font=("Segoe UI", 11, "bold")).grid(row=0, column=0, sticky="w")
        self._legend_row(legend, 1, "✕", PALETTE["x"], "Player X")
        self._legend_row(legend, 2, "◯", PALETTE["o"], "Player O")
        self._legend_row(legend, 3, "⛔", PALETTE["obs"], "Obstacle")

        # Scoreboard
        score = tk.Frame(side_card, bg=PALETTE["panel"], pady=8)
        score.grid(row=8, column=0, sticky="ew")
        tk.Label(score, text="Score", bg=PALETTE["panel"], fg=PALETTE["text"], font=("Segoe UI", 11, "bold")).grid(row=0, column=0, sticky="w")

        self.score_x = tk.IntVar(value=0)
        self.score_o = tk.IntVar(value=0)
        self.score_d = tk.IntVar(value=0)

        self.score_lbl = tk.Label(
            score,
            text=self._score_text(),
            bg=PALETTE["panel"],
            fg=PALETTE["muted"],
            font=("Segoe UI", 10),
            justify="left",
        )
        self.score_lbl.grid(row=1, column=0, sticky="w", pady=(2, 0))

        # Status bar
        self.status = tk.StringVar(value="Click a cell to start.")
        status_bar = ttk.Frame(root, padding=(16, 10), style="App.TFrame")
        status_bar.grid(row=2, column=0, columnspan=2, sticky="ew")
        ttk.Label(status_bar, textvariable=self.status, style="Status.TLabel").grid(row=0, column=0, sticky="w")

        # Build board items
        self.cell_items = {}
        for r in range(SIZE):
            for c in range(SIZE):
                x0 = c * self.step
                y0 = r * self.step
                x1 = x0 + self.cell_px
                y1 = y0 + self.cell_px
                tag = f"cell_{r}_{c}"

                rect = self.canvas.create_rectangle(
                    x0,
                    y0,
                    x1,
                    y1,
                    fill=PALETTE["cell_empty"],
                    outline=PALETTE["border"],
                    width=2,
                    tags=("cell", tag),
                )
                text = self.canvas.create_text(
                    x0 + self.cell_px / 2,
                    y0 + self.cell_px / 2,
                    text="",
                    fill=PALETTE["text"],
                    font=("Segoe UI", 26, "bold"),
                    tags=("cell", tag),
                )
                self.cell_items[(r, c)] = (rect, text)

        # Events
        self.canvas.bind("<Button-1>", self.on_canvas_click)
        self.canvas.bind("<Motion>", self.on_canvas_motion)
        self.canvas.bind("<Leave>", lambda e: self._set_hover(None))

        # Game state
        self.K = K_TO_WIN
        self.total_moves = 0
        self.last_move = None
        self.turn_player = "X"
        self.game_over = False
        self.hover_cell = None

        self._sync_controls()
        self.reset_game()

    # ---------- Sidebar helpers ----------
    def _legend_row(self, parent, row, symbol, color, label):
        box = tk.Frame(parent, bg=PALETTE["panel"])
        box.grid(row=row, column=0, sticky="w", pady=1)
        tk.Label(box, text=symbol, bg=PALETTE["panel"], fg=color, font=("Segoe UI", 12, "bold"), width=2).pack(side="left")
        tk.Label(box, text=label, bg=PALETTE["panel"], fg=PALETTE["muted"], font=("Segoe UI", 10)).pack(side="left")

    def _score_text(self):
        return f"X: {self.score_x.get()}\nO: {self.score_o.get()}\nDraws: {self.score_d.get()}"


    def _sync_controls(self):
        is_ai = self.mode_var.get() == "AI"
        try:
            self.depth_scale.config(state="normal" if is_ai else "disabled")
        except Exception:
            pass

    def reset_scores(self):
        self.score_x.set(0)
        self.score_o.set(0)
        self.score_d.set(0)
        self.score_lbl.config(text=self._score_text())
        self.status.set("Scores reset.")

    # ---------- Board rendering ----------
    def _cell_center(self, r, c):
        return (c * self.step + self.cell_px / 2, r * self.step + self.cell_px / 2)

    def clear_win_line(self):
        self.canvas.delete("winline")

    def draw_win_line(self, path):
        if not path:
            return
        (r1, c1) = path[0]
        (r2, c2) = path[-1]
        x1, y1 = self._cell_center(r1, c1)
        x2, y2 = self._cell_center(r2, c2)
        self.canvas.create_line(
            x1,
            y1,
            x2,
            y2,
            fill=PALETTE["win"],
            width=8,
            capstyle=tk.ROUND,
            tags="winline",
        )
        # Highlight winning cells
        for (rr, cc) in path:
            rect, _ = self.cell_items[(rr, cc)]
            self.canvas.itemconfigure(rect, outline=PALETTE["win"], width=4)

    def update_turn_ui(self):
        if self.turn_player == "X":
            self.turn_text.set("Player X")
            self.turn_dot.itemconfigure(self.turn_dot_id, fill=PALETTE["x"])
        else:
            self.turn_text.set("Player O")
            self.turn_dot.itemconfigure(self.turn_dot_id, fill=PALETTE["o"])

    def update_cell(self, r, c):
        val = self.board[r][c]
        rect, text = self.cell_items[(r, c)]

        if val == EMPTY:
            self.canvas.itemconfigure(rect, fill=PALETTE["cell_empty"], outline=PALETTE["border"], width=2)
            self.canvas.itemconfigure(text, text="")
            return

        if val == OBSTACLE:
            self.canvas.itemconfigure(rect, fill=PALETTE["cell_obs"], outline=PALETTE["border"], width=2)
            self.canvas.itemconfigure(text, text="⛔", fill=PALETTE["obs"], font=("Segoe UI", 22, "bold"))
            return

        if val == "X":
            self.canvas.itemconfigure(rect, fill=PALETTE["cell_x"], outline=PALETTE["border"], width=2)
            self.canvas.itemconfigure(text, text="✕", fill=PALETTE["x"], font=("Segoe UI", 28, "bold"))
            return

        if val == "O":
            self.canvas.itemconfigure(rect, fill=PALETTE["cell_o"], outline=PALETTE["border"], width=2)
            self.canvas.itemconfigure(text, text="◯", fill=PALETTE["o"], font=("Segoe UI", 28, "bold"))
            return

    def refresh_ui(self):
        for r in range(SIZE):
            for c in range(SIZE):
                self.update_cell(r, c)
        self.update_turn_ui()

    # ---------- Hover / click ----------
    def _cell_at_xy(self, x, y):
        if x < 0 or y < 0 or x >= self.board_px or y >= self.board_px:
            return None
        c = int(x // self.step)
        r = int(y // self.step)
        if not (0 <= r < SIZE and 0 <= c < SIZE):
            return None
        # inside the real cell area (not in the gap)
        if (x - c * self.step) > self.cell_px or (y - r * self.step) > self.cell_px:
            return None
        return (r, c)

    def _set_hover(self, rc):
        # restore old hover
        if self.hover_cell is not None:
            r0, c0 = self.hover_cell
            if self.board[r0][c0] == EMPTY:
                rect, _ = self.cell_items[(r0, c0)]
                self.canvas.itemconfigure(rect, fill=PALETTE["cell_empty"], outline=PALETTE["border"], width=2)
        self.hover_cell = rc
        if rc is None:
            return

        r, c = rc
        if self.game_over:
            return
        if self.board[r][c] != EMPTY:
            return
        # prevent hover during AI turn
        if self.mode_var.get() == "AI" and self.turn_player == "O":
            return

        rect, _ = self.cell_items[(r, c)]
        self.canvas.itemconfigure(rect, fill=PALETTE["cell_hover"], outline=PALETTE["accent"], width=3)

    def on_canvas_motion(self, event):
        rc = self._cell_at_xy(event.x, event.y)
        if rc != self.hover_cell:
            self._set_hover(rc)

    def on_canvas_click(self, event):
        if self.game_over:
            return
        if self.mode_var.get() == "AI" and self.turn_player == "O":
            return
        rc = self._cell_at_xy(event.x, event.y)
        if rc is None:
            return
        r, c = rc
        if self.board[r][c] != EMPTY:
            return

        # Human move
        self.make_move(r, c, self.turn_player)

        # AI move (if enabled)
        if (not self.game_over) and self.mode_var.get() == "AI" and self.turn_player == "O":
            self.status.set("AI thinking...")
            # give UI time to update before computation
            self.root.after(60, self.ai_move)

    # ---------- Game logic ----------
    def reset_game(self):
        self.clear_win_line()
        self.board = create_board()

        # Read settings
        self.ai_depth = int(self.ai_depth_var.get())
        self.start_obstacles = int(self.start_obs_var.get())

        init_obstacles(self.board, n=self.start_obstacles, K=self.K)

        self.turn_player = "X"
        self.total_moves = 0
        self.last_move = None
        self.game_over = False

        self.refresh_ui()
        self.status.set("Player X turn.")
        self._set_hover(None)

    def _finish_game(self, message, winner=None, win_path=None):
        self.game_over = True
        self._set_hover(None)
        self.refresh_ui()
        if win_path:
            self.draw_win_line(win_path)
        if winner == "X":
            self.score_x.set(self.score_x.get() + 1)
        elif winner == "O":
            self.score_o.set(self.score_o.get() + 1)
        elif winner is None:
            self.score_d.set(self.score_d.get() + 1)
        self.score_lbl.config(text=self._score_text())
        self.status.set(message)

    def make_move(self, r, c, player):
        self.board[r][c] = player
        self.update_cell(r, c)

        self.total_moves += 1
        self.last_move = (r, c)

        # Win?
        win_path = find_winning_line(self.board, player, K=self.K)
        if win_path:
            self._finish_game(f"Player {player} wins!", winner=player, win_path=win_path)
            return

        # Obstacle?
        freq = obstacle_frequency(self.board)
        if self.total_moves % freq == 0:
            pos = add_obstacle_fair(self.board, last_move=self.last_move, K=self.K, players=("X", "O"))
            if pos is not False:
                rr, cc = pos
                self.update_cell(rr, cc)
                self.status.set(f"Obstacle added at ({rr},{cc})")

        # Draw?
        if is_draw(self.board):
            self._finish_game("Draw!", winner=None)
            return

        # Next turn
        self.turn_player = "O" if player == "X" else "X"
        self.update_turn_ui()

        if not self.status.get().startswith("Obstacle added"):
            self.status.set(f"Player {self.turn_player} turn.")

    def ai_move(self):
        if self.game_over or self.turn_player != "O":
            return

        move = choose_ai_move(
            self.board,
            ai="O",
            human="X",
            depth=self.ai_depth,
            K=self.K,
            total_moves=self.total_moves,
            last_move=self.last_move,
            sample_obstacles=SAMPLE_OBSTACLES,
        )

        if move is None:
            self._finish_game("No legal moves left.")
            return

        r, c = move
        self.make_move(r, c, "O")


def launch_app():
    root = tk.Tk()
    TicTacToeApp(root)
    root.mainloop()


launch_app()

